# Autoregressive Point-Processes as Latent State-Space Models

## Configure notebook

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

# Load scipy/numpy/matplotlib
from   scipy.linalg import expm
import matplotlib.pyplot as plt
from   pylab import *

# Configure figure resolution
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['savefig.dpi'   ] = 100

from izh       import * # Routines for sampling Izhikevich neurons
from plot      import * # Misc. plotting routines
from glm       import * # GLM fitting
from arppglm   import * # Sampling and integration
from utilities import * # Other utilities
from arguments import * # Argument verification

figure_prefix = "RuleSanguinetti2018_figure_"

import numpy as np

In [ ]:
from neurotools.nlab import *

# Case study: phasic bursting Izhikevich neuron

In [ ]:
# Izhikevich parameters
izh = (0.02,0.25,-55,0.05) # a, b, c, d
dt  = 1.0

nplot = 1000   # time points for plotting

# Generate constant drive with synaptic noise
I = 0.6
stimulus = randn(nplot)*sqrt(I)+I

# Plot current input
subplot(311); plot(stimulus)
xlim(0,nplot); nox(); noaxis(); ylabel('pA')
title('Current injected')

# Solve Izh model
state = sim_izh(*izh,stimulus)

# Plot voltage and spikes
subplot(312); plot(state[:,1],color=OCHRE);
xlim(0,nplot); noaxis(); addspikes(state[:,-1]); ylabel('mV');
title('Simulated voltage and spikes');

## Train model on pulses

GLMs can emulate neural firing, but have limited ability to generalize outside of the dynamical regime in which they are trained (Weber & Pillow 2017). For this reason, we train with stimuli that elicit phasic bursting responses (tonic bursting seems to be possible at higher stimulation currents, but interferes with the GLMs ability to model the phasic bursting regime).

In [ ]:
offset1  = 0
offset2  = 0
amp1     = 2.5
amp2     = 1.5
duration1= 200
duration2= 200
stimlen  = 400
stimat   = 125
stim1    = np.zeros(stimlen) + offset1
stim2    = np.zeros(stimlen) + offset2
stim1[stimat:stimat+duration1] = amp1
stim2[stimat:stimat+duration2] = sample_ou_process(0,amp2,20,1,duration2).ravel()

stim1,stim2 = stim2,stim1

In [ ]:
figure(figsize=(10,8))
ntrain     = len(stim1)

subplot2grid((5,2),(0,0))
plot(stim1,color=BLACK)
plot(stim2,color=RUST)
xlim(0,ntrain)
simpleraxis()

# Solve Izh model and get voltage and spikes
state = sim_izh(*izh,stim1,dt=dt)
v,Ytrain = state[:,1],state[:,2]

# Plot voltage and spikes
subplot2grid((5,2),(1,0),colspan=2)
plot(v,color=OCHRE); addspikes(Ytrain,lw=0.05);
noaxis(); xlim(0,ntrain); ylabel('mV');

# Define history basis functions
N = 150   # Duration of history filter
K = 8     # number of basis elements
D = 5     # Duration of shortest basis element
B = make_cosine_basis(K,N,D,normalize=False)

# Plot history basis functions
subplot2grid((5,2),(0,1))
plot(B.T,color=BLACK,clip_on=False);
xlim(0,N); ylim(0,0.5); simpleaxis()

### Generate stimulus and spiking history training features

# Build stimulus filter (history trace of I)
# These are needed to model subthreshold dynamics
Bh1 = array([convolve(b,stim1) for b in B]).T[:ntrain]
Bh2 = array([convolve(b,stim2) for b in B]).T[:ntrain]
Bp = concatenate([zeros((K,1)),B],axis=1)
By1 = array([convolve(b,Ytrain) for b in Bp]).T[:ntrain]

## Train model
# Compose feature vector and fit GLM
X = concatenate([By1,Bh1],axis=1)
m,bhat = fitGLM(X,Ytrain)
bhat_spikehist = bhat[:K]
bhat_stimulus  = bhat[K:]
beta = bhat[:K].reshape(K,1)

# Check stability via sampling
subplot2grid((5,2),(2,0),colspan=2)
st1 = (m + Bh1.dot(bhat_stimulus))
#y,l  = ensemble_sample(st1,B,beta,M=100)
y,l  = ensemble_sample_continuous(st1,B,beta,M=50,oversample=200)
pcolormesh(-int32(y.T>0),cmap='gray')
simpleraxis()

# Solve Izh model and get voltage and spikes
subplot2grid((5,2),(3,0),colspan=2)
state = sim_izh(*izh,stim2,dt=dt)
v,Ytest = state[:,1],state[:,2]
By2 = array([convolve(b,Ytest) for b in Bp]).T[:ntrain]
plot(v,color=OCHRE); addspikes(Y,lw=0.05);
noaxis(); xlim(0,ntrain); ylabel('mV');
title('Simulated voltage and spikes');

subplot2grid((5,2),(4,0),colspan=2)
st2 = (m + Bh2.dot(bhat_stimulus))
y,l  = ensemble_sample_continuous(st2,B,beta,M=50,oversample=200)
pcolormesh(-int32(y.T>0),cmap='gray')
simpleraxis()

tight_layout()

In [ ]:
saved_training_model = {}
saved_training_model['K'] = K
saved_training_model['B'] = B
saved_training_model['By'] = By
saved_training_model['Bh'] = Bh1
saved_training_model['A'] = A
saved_training_model['C'] = C
saved_training_model['Y'] = Ytrain
saved_training_model['dt'] = dt
saved_training_model['Bh_train'] = Bh1
saved_training_model['By_train'] = By1
saved_training_model['Y_train'] = Ytrain
saved_training_model['Bh_test'] = Bh2
saved_training_model['By_test'] = By2
saved_training_model['Y_test'] = Ytest
saved_training_model = scipy.io.savemat('saved_training_model_badburster.mat',saved_training_model)